In [1]:
import psycopg2
from psycopg2  import extras
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

DATABASE = 'postgresql'
USER = 'root'
PASSWORD = 'root'
HOST = 'postgres' # shared-networkに所属しているpostgres containerを指定できる
PORT = '5432'
DB_NAME = 'sandbox'

CONNECT_STR = '{}://{}:{}@{}:{}/{}'.format(DATABASE, USER, PASSWORD, HOST, PORT, DB_NAME)
engine = create_engine(CONNECT_STR)

conn = engine.connect()
res = conn.execute("select datname, datdba, encoding, datcollate, datctype from pg_database;")

for row in res:
    print(row)

('postgres', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('sandbox', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template1', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template0', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')


In [2]:
# create poptbl
tablename = 'x002_poptbl2'
q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename)
res = conn.execute(q1)

q2 = """
CREATE TABLE  {} (
    pref_name  varchar(256) NOT NULL,
    sex integer NOT NULL,
    population integer NOT NULL
);
""".format(tablename)

if sum(1 for _ in res) == 0:
    print("create")
    conn.execute(q2)
    res = conn.execute(q1)
    sum(1 for _ in res)

In [3]:
# insert data
data = (
    ('徳島', 1, 60),
    ('徳島', 2, 40),
    ('香川', 1, 100),
    ('香川', 2, 100),
    ('愛媛', 1, 100),
    ('愛媛', 2, 50),
    ('高知', 1, 100),
    ('高知', 2, 100),
    ('福岡', 1, 100),
    ('福岡', 2, 200),
    ('佐賀', 1, 20),
    ('佐賀', 2, 80),
    ('長崎', 1, 125),
    ('長崎', 2, 125),
    ('東京', 1, 250),
    ('東京', 2, 150),
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
            Column('pref_name', String, primary_key=False),
            Column('sex', Integer),
            Column('population', Integer),
           )

for i in data:
    ins = tbl.insert().values(pref_name=i[0], sex=i[1], population=i[2])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

('徳島', 1, 60)
('徳島', 2, 40)
('香川', 1, 100)
('香川', 2, 100)
('愛媛', 1, 100)
('愛媛', 2, 50)
('高知', 1, 100)
('高知', 2, 100)
('福岡', 1, 100)
('福岡', 2, 200)
('佐賀', 1, 20)
('佐賀', 2, 80)
('長崎', 1, 125)
('長崎', 2, 125)
('東京', 1, 250)
('東京', 2, 150)


期待する結果

|県名|男|女|
|:--|:--|:--|
|佐賀|20|80|
|徳島|60|40|
|愛媛|100|50|
|香川|100|100|
|高知|100|100|
|福岡|100|200|
|長崎|125|125|
|東京|250|150|

In [4]:
q = f"""
select 
    pref_name,
    sum(
        case 
        when sex = 1 then population else 0 end
    ) as 男,
    sum(
        case
        when sex = 2 then population else 0 end
    ) as 女
    from {tablename}
    group by pref_name
    order by 男;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select 
    pref_name,
    sum(
        case 
        when sex = 1 then population else 0 end
    ) as 男,
    sum(
        case
        when sex = 2 then population else 0 end
    ) as 女
    from x002_poptbl2
    group by pref_name
    order by 男;

('佐賀', 20, 80)
('徳島', 60, 40)
('愛媛', 100, 50)
('香川', 100, 100)
('高知', 100, 100)
('福岡', 100, 200)
('長崎', 125, 125)
('東京', 250, 150)
